# 텍스트 기반 추천

## 필요 라이브러리 

In [1]:
import pandas as pd
import numpy as np
import warnings;

import torch

warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

 __ai허브데이터 xlsx라 csv로 변환과정__ 
!pip install xlrd

!pip install openpyxl

xlsx = pd.read_excel('C:/Users/namva/자연어/dataset/감성대화/원천데이터//sentimenttalk.xlsx')

xlsx.to_csv('C:/Users/namva/자연어/dataset/감성대화/원천데이터//sentimenttalk.xlsx')

In [2]:
import torch
from transformers import AutoTokenizer, AutoModel, BertTokenizer

## 모델 불러오기
tokenizer = BertTokenizer.from_pretrained("beomi/kcbert-large")

In [22]:
def Build_X (sents, tokenizer, device):
    X = tokenizer(sents, padding=True, truncation=True, return_tensors='pt')
    return torch.stack([
        X['input_ids'],
        X['token_type_ids'],
        X['attention_mask']
    ], dim=1).to(device)

device = torch.device('cpu')
x = '우리 집에서 고양이 보고갈래?'
x = Build_X(x, tokenizer, device)
input_ids = x[:, 0]
print(input_ids)

tensor([[    2,  7983, 10103, 19490,  8198,  4320,  4369,    32,     3]])


In [3]:
import pandas as pd
data=pd.read_csv('sentimenttalk.xlsx')
df=data[ ['감정_대분류','감정_소분류','사람문장1','기계문장1','사람문장2','기계문장2','사람문장3','기계문장3']]

# 감정과 사람의 문장만 있게 변경
person = df[['감정_대분류','감정_소분류','사람문장1','사람문장2','사람문장3']]

In [4]:
person.head(2)

,감정_대분류,감정_소분류,사람문장1,사람문장2,사람문장3
0,기쁨,감사하는,오늘 반장 선거에서 내가 반장이 됐어! 친구들이 날 믿어줘서 너무 고맙다.,친구들에게 어떻게 보답하면 좋을지 고민이야.,친구 한 명씩 찾아가서 고맙다고 말하고 바라는 점을 물어보면 좋을 것 같아.
1,기쁨,감사하는,오늘 시험에 친구가 알려준 문제가 나왔어. 결과가 좋아서 기쁘다.,말로 하기 쑥스러운데.,같이 떡볶이 먹고 피시방 가자고 해야겠어.


In [7]:
sentence = '우리집 고양이 나만없어'
new_data = [0,0,sentence,0,0]
person.loc[len(person)] = new_data

In [8]:
person.tail()

,감정_대분류,감정_소분류,사람문장1,사람문장2,사람문장3
9996,상처,충격받은,요즘 두통이 잦았는데 결국 이런 일이 생겼네. 난 건강한 줄 알았는데 충격받았어.,두통 때문에 병원에 갔는데 뇌척수막염 치료 시기를 놓쳤대. 눈앞이 캄캄해.,슬프지만 받아들이고 아내한테 이야기해야겠지.
9997,불안,걱정스러운,친구가 돈이 행복의 전부가 아니라고 하더니 노후 준비를 전혀 안 한 거야. 어휴 정...,여태 노후 준비를 하나도 안 해놓다니 어쩌려고 그러는지 모르겠어.,친구에게 돈이 중요하다고 다시 한번 충고해줘야 할 것 같아.
9998,불안,두려운,아들이 갑자기 찾아왔는데 손자가 희귀병이래. 너무 두려워.,아직 치료법도 없대. 아직 어린데 입원해 있을 생각을 하니 눈물이 나.,손자 입원하면 자주 찾아가고 간호해야지.
9999,당황,남의 시선을 의식하는,친구들에게는 자식이 선물해 줬다고 했지만 사실은 다 내가 구입한 거야.,자식들에게 좋은 대접을 받는다고 말하고 싶어서 거짓말을 했던 것 같아.,일단은 가까운 친구들에게라도 솔직하게 털어놓도록 노력해 봐야겠어.
10000,0,0,우리집 고양이 나만없어,0,0


In [15]:
docs = person['사람문장1']
sents = [sent for sent in docs]
device = torch.device('cpu')

x = Build_X(sents, tokenizer, device)

In [16]:
input_ids = x[:, 0]
x = input_ids.numpy()
x = np.asmatrix(x)
print(x)
print(type(x))

[[    2  8564  1483 ...     0     0     0]
 [    2  8564 10897 ...     0     0     0]
 [    2  8564  8727 ...     0     0     0]
 ...
 [    2 17831  9120 ...     0     0     0]
 [    2 12404  9548 ...     0     0     0]
 [    2 21192 10799 ...     0     0     0]]
<class 'numpy.matrix'>


## 코사인 유사도 및 데이터 변환

In [17]:
print(x[:3])

[[    2  8564  1483  4099 15575  8227  1483  8461   896  4071     5 21192
   4017   595 13106 10579  8069 12850    17     3     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0]
 [    2  8564 10897  4113 17831  9822  4152  8732  9069  4071    17 12300
  13492 29801  4020    17     3     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0]
 [    2  8564  8727 14716  3167  8920  9647  4071    17 14392  8069 23958
   8203  8616 15037   601  4071    17     3     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0]]


In [18]:
cosine_mat= cosine_similarity(x, x) # tf-idf 유사도 -> cosine 유사도
np.round(cosine_mat, 4) 

#index기반 구하기
idx2title={}
for i,c in enumerate(person['사람문장1']): idx2title[i] = c
# title기반 구하기
title2idx={}
for i,c in idx2title.items(): title2idx[c]= i


In [19]:
print(cosine_mat)
title2idx

[[1.         0.54913733 0.71249256 ... 0.53383704 0.45309508 0.62226278]
 [0.54913733 1.         0.64055847 ... 0.43663829 0.77741368 0.70023274]
 [0.71249256 0.64055847 1.         ... 0.68756334 0.62992521 0.72809137]
 ...
 [0.53383704 0.43663829 0.68756334 ... 1.         0.57679463 0.66266169]
 [0.45309508 0.77741368 0.62992521 ... 0.57679463 1.         0.73929346]
 [0.62226278 0.70023274 0.72809137 ... 0.66266169 0.73929346 1.        ]]


{'오늘 반장 선거에서 내가 반장이 됐어! 친구들이 날 믿어줘서 너무 고맙다.': 0,
 '오늘 시험에 친구가 알려준 문제가 나왔어. 결과가 좋아서 기쁘다.': 1,
 '오늘 대학 합격 통지를 받았어. 엄마가 너무 기뻐하는데 나는 눈물이 났어.': 2,
 '오늘 졸업식인데 선생님께서 아프셔서 오지 못하셨어. 그동안 감사하다는 말도 못했는데.': 3,
 '나 올림피아드에서 금상 받았어! 그동안 고생이 헛되지 않아서 너무 기뻐.': 4,
 '요즘 부모님 부부싸움 때문에 많이 힘들었는데 내 친구 미영이 덕분에 기분이 한결 나아졌어.': 5,
 '오늘 학교에서 친구랑 다퉜는데 선생님은 나만 뭐라고 하셔.': 6,
 '오늘 준비물을 깜빡 했는데 민우가 쉬는 시간에 뛰어가서 사다 준 거 있지. 감동이야.': 7,
 '오늘 발표시간에 내 차례가 되니 눈앞이 캄캄해지는 거 있지.': 8,
 '태희는 천재인 거 같아. 매번 막히는 문제를 물어보면 모르는 게 없어!': 9,
 '담임선생님에게 고민을 이야기하면 마음이 편해져.': 10,
 '대학에서 뭘 공부하고 커서 뭘 해야 할지 모르겠어.': 11,
 '선생님께서 빨리 달려와 주셔서 그 애들이 날 못 때렸어.': 12,
 '학교 폭력을 당하고 있었는데 선생님께서 그걸 먼저 알아채고 도와주셨어.': 13,
 '선생님이 나를 괴롭힌 가해자들 뿐만 아니라 방관자들도 혼을 내셔서 속이 시원해': 14,
 '이번 학교 폭력 위원회에서 가해자들이 징계를 크게 받게 됐어.': 15,
 '아까 내가 혼자 울고 있을 때 담임 선생님께서 손수건을 건네 주셔서 감동했어.': 16,
 '학교 폭력 상담 센터 분들이 모두 나를 다정하고 조심스럽게 대해 주셨어.': 17,
 '보건 선생님이 내가 양호실에 자주 찾아오는 걸 보고 학교 폭력 신고를 해 주셨어.': 18,
 '선생님 덕분에 날 괴롭히던 애들과 오해를 풀고 잘 지낼 수 있게 됐어. 정말 감사드려.': 19,
 '내가 따돌림 당하고 있다는 걸 선생님께 말씀드리고 났더니 훨씬 편해졌어.': 20

## 실행과정(입력 -> 유사도 -> 상위 10개의 유사도 출력)

In [21]:
# 입력
# new_sen = '학교 폭력 상담 센터 분들이 모두 나를 다정하고 조심스럽게 대해 주셨어.'
# x = Build_X(new_sen, tokenizer, device)
# input_ids = x[:, 0]
# x = input_ids.numpy()
# x = np.asmatrix(x)


idx= title2idx['오빠 언제와']
# 동일한 텍스트 제외 유사도 구하기, 정렬
sim_score = [(i,c) for i,c in enumerate(cosine_mat[idx]) if i != idx]
sim_score = sorted(sim_score, key=lambda x:x[1],reverse=True)
# 상위 10개 추출
sim_score = [(idx2title[i],c) for i,c in sim_score[:10]]
sim_score


KeyError: '오빠 언제와'